# Dataset
https://www.kaggle.com/mrisdal/fake-news#fake.csv

Example domain classifications (in flux) include:

  * **Fake News**: Sources that fabricate stories out of whole cloth with the intent of pranking the public.
  * **Satire**: Sources that provide humorous commentary on current events in the form of fake news.
  *  **Extreme Bias**: Sources that traffic in political propaganda and gross distortions of fact.
  *  **Conspiracy Theory**: Sources that are well-known promoters of kooky conspiracy theories.
  *  **Rumor Mill**: Sources that traffic in rumors, innuendo, and unverified claims.
  *  **State News**: Sources in repressive states operating under government sanction.
  *  **Junk Science**: Sources that promote pseudoscience, metaphysics, naturalistic fallacies, and other scientifically dubious claims.
  *  **Hate Group**: Sources that actively promote racism, misogyny, homophobia, and other forms of discrimination.
  *  **Clickbait:** Sources that are aimed at generating online advertising revenue and rely on sensationalist headlines or eye-catching pictures.
  *  **Proceed With Caution**: Sources that may be reliable but whose contents require further verification.
  *  The labels of **bs** and **junksci**, etc. do not constitute capital "t" Truth



In [67]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import mlutils
%matplotlib inline
import pandas as pd
import tensorflow_hub as hub 
print tf.__version__

1.9.0


## Descarga, arregla e inspecciona el dataset Fake news

In [19]:
import os
!mkdir -p data
if not os.path.exists("dataFake/fake.csv"):
    !wget -c -nc https://github.com/chdorrego/tf/raw/master/fake-news.zip -P dataFake
    !unzip dataFake/fake-news.zip -d dataFake
    with open("dataFake/fake.csv", "r") as f:
        lines = f.readlines()
        lines = [i for i in lines if len(i.replace("\n", ""))>0]
    with open("dataFake/fake.csv", "w") as f:
        f.writelines(lines) 

--2018-09-01 08:37:15--  https://github.com/chdorrego/tf/raw/master/fake-news.zip
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chdorrego/tf/master/fake-news.zip [following]
--2018-09-01 08:37:16--  https://raw.githubusercontent.com/chdorrego/tf/master/fake-news.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.220.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.220.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21412001 (20M) [application/zip]
Saving to: ‘dataFake/fake-news.zip’

fake-news.zip       100%[===================>]  20.42M  1.87MB/s    in 13s     

2018-09-01 08:37:29 (1.57 MB/s) - ‘dataFake/fake-news.zip’ saved [21412001/21412001]

Archive:  dataFake/fake-news.zip
  inflating: dataFake/fake.csv    

In [20]:
!head -5 dataFake/fake.csv

"uuid","ord_in_thread","author","published","title","text","language","crawled","site_url","country","domain_rank","thread_title","spam_score","main_img_url","replies_count","participants_count","likes","comments","shares","type"
"6a175f46bcd24d39b3e962ad0f29936721db70db",0,"Barracuda Brigade","2016-10-26T21:41:00.000+03:00","Muslims BUSTED: They Stole Millions In Gov’t Benefits","Print They should pay all the back all the money plus interest. The entire family and everyone who came in with them need to be deported asap. Why did it take two years to bust them? 
Here we go again …another group stealing from the government and taxpayers! A group of Somalis stole over four million in government benefits over just 10 months! 
We’ve reported on numerous cases like this one where the Muslim refugees/immigrants commit fraud by scamming our system…It’s way out of control! More Related","english","2016-10-27T01:49:27.168+03:00","100percentfedup.com","US",25689,"Muslims BUSTED: They Stole Millio

In [77]:
CSV_COLUMNS  = ['uuid','ord_in_thread','author','published','title','text','language','crawled','site_url','country','domain_rank','thread_title','spam_score','main_img_url','replies_count','participants_count','likes','comments','shares','type_label']
#CSV_DEFAULTS = [[''],[0],[''],[''],[''],[''],[''],[''],[''],[''],[0],[''],[-1],[''],[-1],[-1],[-1],[-1],[-1],['']]
CSV_DEFAULTS = [[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],[''],['']]


type_label =  ['bias','conspiracy','hate','state','junksci','satire','fake']
no_valid_labels = ['bs']

fake= pd.read_csv("dataFake/fake.csv", names = CSV_COLUMNS,skiprows=1)
print fake.shape
fake.head()

(12999, 20)


,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type_label
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


### Labels

* bs            10919 
* conspiracy      412 
- bias            354 
- hate            246 
- state           121 
- junksci         102 
- satire          100 
- fake             19


## Realizamos la eleminiación de la clase bs debido a la definición del problema y creamos los archivos test y train

In [58]:
d = fake.loc[fake['type_label'].isin(type_label)]
msk = np.random.rand(len(d)) < 0.8

d_train = d[msk==True]
d_test =  d[msk==False]

d_train.to_csv('dataFake/fake_train.csv', sep=';', encoding='utf-8', header=False)
d_test.to_csv('dataFake/fake_test.csv', sep=';', encoding='utf-8', header=False)
print d_train.count()
print d_test.count()

uuid                  1209
ord_in_thread         1209
author                 832
published             1209
title                 1120
text                  1172
language              1209
crawled               1209
site_url              1209
country               1209
domain_rank           1047
thread_title          1209
spam_score            1209
main_img_url           960
replies_count         1209
participants_count    1209
likes                 1209
comments              1209
shares                1209
type_label            1209
dtype: int64
uuid                  298
ord_in_thread         298
author                196
published             298
title                 280
text                  289
language              298
crawled               298
site_url              298
country               298
domain_rank           272
thread_title          298
spam_score            298
main_img_url          233
replies_count         298
participants_count    298
likes                 298
comme

## Creamos la funcion para la generacion del `input pipeline`

In [87]:
#occupations = list(np.unique(d.occupation))
type_label =  ['bias','conspiracy','hate','state','junksci','satire','fake']



def parse_csv(rows_string_tensor, use_features):
    columns = tf.decode_csv(rows_string_tensor, record_defaults=CSV_DEFAULTS, field_delim=";")
    features = dict(zip(CSV_COLUMNS, columns))

        
    label = features.pop("type_label")
    
    return {k:v for k,v in features.iteritems() if k in use_features}, label


"""
def get_dataset(filename, repeat, batch_size, shuffle):
    use_features  = ['title','type_label']
    
    dataset = tf.data.TextLineDataset(filename)
    dataset = dataset.repeat(repeat)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=300)
    dataset = dataset.map(lambda x: parse_csv(x, use_features))    
    dataset = dataset.batch(batch_size)
    return dataset



def get_features_defs():
    #title = tf.feature_column.numeric_column("title")
    #text = tf.feature_column.numeric_column('text')
    embedded_title = hub.text_embedding_column(
        key="title", 
        module_spec="https://tfhub.dev/google/nnlm-en-dim128/1")
    
    return [title] 
    #return [title,text]

def make_data_iterator(filename, repeat, batch_size, shuffle=True):
    
    dataset = get_dataset(filename, repeat, batch_size, shuffle)
    features_defs = get_features_defs()
    
    features, label = dataset.make_one_shot_iterator().get_next()
    
    next_X = tf.feature_column.input_layer(features, features_defs)
    next_y = label, (-1,1)
    
    return next_X, next_y


tf.reset_default_graph()

features, label = make_data_iterator("dataFake/fake_train.csv", repeat=1, batch_size=10, shuffle=False)
with tf.Session() as sess:
    tf.tables_initializer().run()
    f,l = sess.run([features, label])   

"""  

'\ndef get_dataset(filename, repeat, batch_size, shuffle):\n    use_features  = [\'title\',\'type_label\']\n    \n    dataset = tf.data.TextLineDataset(filename)\n    dataset = dataset.repeat(repeat)\n    if shuffle:\n        dataset = dataset.shuffle(buffer_size=300)\n    dataset = dataset.map(lambda x: parse_csv(x, use_features))    \n    dataset = dataset.batch(batch_size)\n    return dataset\n\n\n\ndef get_features_defs():\n    #title = tf.feature_column.numeric_column("title")\n    #text = tf.feature_column.numeric_column(\'text\')\n    embedded_title = hub.text_embedding_column(\n        key="title", \n        module_spec="https://tfhub.dev/google/nnlm-en-dim128/1")\n    \n    return [title] \n    #return [title,text]\n\ndef make_data_iterator(filename, repeat, batch_size, shuffle=True):\n    \n    dataset = get_dataset(filename, repeat, batch_size, shuffle)\n    features_defs = get_features_defs()\n    \n    features, label = dataset.make_one_shot_iterator().get_next()\n    \n  

In [86]:
dataset = tf.data.TextLineDataset("dataFake/fake_train.csv")
dataset = dataset.batch(10)
use_features  = ['title','type_label']
dataset = dataset.map(lambda x: parse_csv(x, use_features))     
features = dataset.make_one_shot_iterator().get_next()
with tf.Session() as sess:
    print sess.run(features)

InvalidArgumentError: Quoted field has to end with quote followed by delim or end
	 [[Node: DecodeCSV = DecodeCSV[OUT_TYPE=[DT_STRING, DT_STRING, DT_STRING, DT_STRING, DT_STRING, ..., DT_STRING, DT_STRING, DT_STRING, DT_STRING, DT_STRING], field_delim=";", na_value="", select_cols=[], use_quote_delim=true](arg0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0)]]
	 [[Node: IteratorGetNext_7 = IteratorGetNext[output_shapes=[[?], [?]], output_types=[DT_STRING, DT_STRI

In [83]:
dataset = tf.data.TextLineDataset("dataFake/fake_train.csv")
dataset = dataset.batch(10)
features = dataset.make_one_shot_iterator().get_next()
with tf.Session() as sess:
    print sess.run(features)


['0;6a175f46bcd24d39b3e962ad0f29936721db70db;0;Barracuda Brigade;2016-10-26T21:41:00.000+03:00;Muslims BUSTED: They Stole Millions In Gov\xe2\x80\x99t Benefits;"Print They should pay all the back all the money plus interest. The entire family and everyone who came in with them need to be deported asap. Why did it take two years to bust them? '
 'Here we go again \xe2\x80\xa6another group stealing from the government and taxpayers! A group of Somalis stole over four million in government benefits over just 10 months! '
 'We\xe2\x80\x99ve reported on numerous cases like this one where the Muslim refugees/immigrants commit fraud by scamming our system\xe2\x80\xa6It\xe2\x80\x99s way out of control! More Related";english;2016-10-27T01:49:27.168+03:00;100percentfedup.com;US;25689.0;Muslims BUSTED: They Stole Millions In Gov\xe2\x80\x99t Benefits;0.0;http://bb4sp.com/wp-content/uploads/2016/10/Fullscreen-capture-10262016-83501-AM.bmp.jpg;0;1;0;0;0;bias'
 '1;2bdc29d12605ef9cf3f09f9875040a7113b